# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기

## Face Detection vs Face Recognition

패키지설치
> pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp


# 얼굴을 찾고 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils  # 얼굴의 특징을 그리기위한 drawing_utils 모듈 사용


# For webcam input:
# cap = cv2.VideoCapture(0)
# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png')  # 100 X 100
image_left_eye = cv2.imread('left_eye.png')  # 100 X 100
image_nose = cv2.imread('nose.png')  # 300 X 100 (가로 X 세로)


# model_selection > 2가지 중 하나를 고를 수 있음(0or1) 0은 카메라~2미터 근거리 1은 5미터이내의 얼굴에 적합
# min_detection_confidence > 몇 퍼센트 정도되면 얼굴로 인식할것인지
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.9) as face_detection:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            # continue
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.detections:
            # 6개의 특징 : 오른 눈, 왼 눈, 코 끝, 입 중심, 오른 귀, 왼 귀
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]  # 오른 눈
                left_eye = keypoints[1]  # 왼 눈
                nose_tip = keypoints[2]  # 코

                h, w, _ = image.shape  # height, width channel : 이미지로부터 세로,가로,크기 가져옴
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)  # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)  # 초록색
                # # 코에 동그라미
                # cv2.circle(image, nose_tip, 35, (0, 0, 255), 10, cv2.LINE_AA) # 빨간색

                # 각 특징에다 이미지 그리기
                image[right_eye[1]-50 : right_eye[1]+50, right_eye[0]-50 : right_eye[0]+50] = image_right_eye
                image[left_eye[1]-50 : left_eye[1]+50, left_eye[0]-50 : left_eye[0]+50] = image_left_eye
                image[nose_tip[1]-50 : nose_tip[1]+50, nose_tip[0]-150 : nose_tip[0]+150] = image_nose


        # Flip the image horizontally for a selfie-view display.
        # cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        cv2.imshow('MediaPipe Face Detection',cv2.resize(image, None, fx=0.5, fy=0.5))

        # if cv2.waitKey(5) & 0xFF == 27:
        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


> https://opencv-python.readthedocs.io/en/latest/
참고자료
